In [4]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [6]:
import cv2
face_cap = cv2.CascadeClassifier("C:/Users/biswa/AppData/Local/Programs/Python/Python313/Lib/site-packages/cv2/data/haarcascade_frontalface_default.xml")
video_cap = cv2.VideoCapture(0)

while True:
    ret, video_data = video_cap.read() 
    if not ret:
        print("Failed to grab frame")
        break
    gray = cv2.cvtColor(video_data, cv2.COLOR_BGR2GRAY)
    faces = face_cap.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)
    for (x, y, w, h) in faces:
        cv2.rectangle(video_data, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.imshow("Video_live", video_data)
    
    if cv2.waitKey(10) == ord("a"):
        break
video_cap.release()
cv2.destroyAllWindows()


In [4]:
import cv2


def facebox(facenet, frame):
    framewidth = frame.shape[1]
    frameheight = frame.shape[0]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (277, 277), [104, 117, 123], swapRB=False)
    facenet.setInput(blob)
    detection = facenet.forward()
    bbox = []
    print(detection.shape)
    for i in range(detection.shape[2]):
        confidence = detection[0, 0, i, 2]
        if confidence > 0.7:
            x1 = int(detection[0, 0, i, 3] * framewidth)
            y1 = int(detection[0, 0, i, 4] * frameheight)
            x2 = int(detection[0, 0, i, 5] * framewidth)
            y2 = int(detection[0, 0, i, 6] * frameheight)
            bbox.append([x1, y1, x2, y2])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)
    return frame, bbox



faceproto ="C:/Users/biswa/PROJECT/Gender detection/opencv_face_detector.pbtxt"
facemodel ="C:/Users/biswa/PROJECT/Gender detection/opencv_face_detector_uint8.pb"
ageproto="C:/Users/biswa/PROJECT/Gender detection/age_deploy.prototxt"
agemodel="C:/Users/biswa/PROJECT/Gender detectio/age_net.caffemodel"
genderproto="C:/sers/biswa/PROJECT/Gender detection/gender_deploy.prototxt"
gendermodel="C:/Users/biswa/PROJECT/Gender detection/gender_net.caffemodel"


MODE_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
agelist = ['(0-2)', '(3-5)', '(6-8)', '(9-11)', '(12-14)', '(15-20)', '(20-32)', '(38-43)', '(48-50)', '(52-98)', '(99-100)']
genderlist = ['Male', 'Female']


facenet = cv2.dnn.readNet(faceproto, facemodel)
agenet = cv2.dnn.readNet(ageproto, agemodel)
gendernet = cv2.dnn.readNet(genderproto, gendermodel)


video_cap = cv2.VideoCapture(0)
while True:
    ret, frame = video_cap.read()
    if not ret:
        print("Error: Unable to read from camera.")
        break
    
    framenet, bbox = facebox(facenet, frame)
    for box in bbox:
        face = frame[box[1]:box[3], box[0]:box[2]]
        if face.size == 0:
            continue  

        
        face_resized = cv2.resize(face, (227, 227))
        blob = cv2.dnn.blobFromImage(face_resized, 1.0, (227, 227), MODE_MEAN_VALUES, swapRB=False)

        
        gendernet.setInput(blob)
        gender_pred = gendernet.forward()
        gender = genderlist[gender_pred[0].argmax()]

        
        agenet.setInput(blob)
        age_pred = agenet.forward()
        age = agelist[age_pred[0].argmax()]

    
        label = "{}.{}".format(gender, age)
        cv2.putText(frame, label, (box[0], box[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1.2, (255, 255, 255), 2)

    cv2.imshow("Age-Gender Detection", frame)
    if cv2.waitKey(10) == ord("a"):
        break
        
video_cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\caffe\caffe_io.cpp:1138: error: (-2:Unspecified error) FAILED: fs.is_open(). Can't open "C:/Users/biswa/PROJECT/Gender detectio/age_net.caffemodel" in function 'cv::dnn::ReadProtoFromBinaryFile'
